In [1]:
import pandas as pd
stock_list = pd.read_csv("../database/yearly/tw_stock_list.csv")
s = stock_list.sort_values(by="volume", ascending=False)
s.head()

,Unnamed: 0,ticker,codename,price,volume,volume_ntd,ticker.1,codename.1,market,listed,TW_TWO
1827,2095,1314,中石化,11.25,197173,22.182,1314,中石化,1314.TW,True,TW
1115,1213,2618,長榮航,35.55,146037,51.916,2618,長榮航,2618.TW,True,TW
1646,1870,2515,中工,17.70,129256,22.878,2515,中工,2515.TW,True,TW
1058,1133,2501,國建,38.30,128465,49.202,2501,國建,2501.TW,True,TW
706,720,2363,矽統,61.80,124192,76.751,2363,矽統,2363.TW,True,TW


In [11]:
from functions import data_strat2, nav_returns_backtest_only
from datetime import datetime, timedelta
strat = "strat2"
stock_list = pd.read_csv("../database/yearly/tw_stock_list.csv")
for start_year in [2020, 2021, 2022, 2023, 2024]:
    start = f"{start_year}-01-01"
    end = f'{start_year+1}-01-01'
    returns = []
    winrates = []
    avg_ret = []
    trade_total = []
    last_open_date = []
    last_open_price = []
    tickers = []
    df_final = pd.DataFrame()
    for ticker in s['market'][:500]:
        ticker_bar = ticker.replace(".", "_")
        df = data_strat2(ticker_bar, start, end)

        return_total, winrate, avg_return, trades, last_openposition_date, last_openposition_price = nav_returns_backtest_only(df)
        returns.append(return_total)
        winrates.append(winrate)
        avg_ret.append(avg_return)
        trade_total.append(trades)
        tickers.append(ticker[:4])
        # last_open_date.append(df.index[last_openposition_date])
        last_open_price.append(last_openposition_price)
    df_final['ticker'] = tickers
    df_final['return_total'] = returns
    df_final['winrate'] = winrates
    df_final['avg_return'] = avg_ret
    df_final['trade_total'] = trade_total
    # df_final['last_openposition_date'] = last_open_date
    df_final['last_openposition_price'] = last_open_price
    df_final.to_csv(f"backtest_by_year/{strat}_{start_year}_{start_year+1}.csv")

c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\cwcha\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\cwcha\AppData\Loca

In [12]:
import numpy as np
max_returns = []
min_returns = []
starts = []
return_thresholds = []
stock_list_len = []
mean_returns = []
trade_totals = []
mean_winrates = []
final = pd.DataFrame()
for start in [2020, 2021, 2022, 2023]:
    df_final = pd.read_csv(f"backtest_by_year/{strat}_{start}_{start+1}.csv")
    df_outsample = pd.read_csv(f"backtest_by_year/{strat}_{start+1}_{start+2}.csv")
    for return_threshold in [0.1, 0.2, 0.3, 0.4, 0.5]:
        return_1 = list(df_final[df_final['return_total'] > return_threshold]['ticker'])

        return_5_avg = df_outsample[df_outsample['ticker'].isin(return_1)]

        max_return = np.max(return_5_avg['return_total'])
        min_return = np.min(return_5_avg['return_total'])
        mean_return = np.mean(return_5_avg['return_total'])
        mean_winrate = np.mean(return_5_avg['winrate'])
        trade_total = np.sum(return_5_avg['trade_total'])
        max_returns.append(max_return)
        min_returns.append(min_return)
        return_thresholds.append(return_threshold)
        starts.append(start)
        stock_list_len.append(len(return_1))
        mean_returns.append(mean_return)
        trade_totals.append(trade_total)
        mean_winrates.append(mean_winrate)
final['starts'] = starts
final['max_returns'] = max_returns
final['min_returns'] = min_returns
final["return_thresholds"] = return_thresholds
final["stock_list_len"] = stock_list_len
final["mean_returns"] = mean_returns
final["trade_totals"] = trade_totals
final["mean_winrates"] = mean_winrates
final.sort_values(by='mean_returns', ascending=False)
# final.sort_values(by="return_thresholds", ascending=False)

,starts,max_returns,min_returns,return_thresholds,stock_list_len,mean_returns,trade_totals,mean_winrates
13,2022,0.655935,-0.208488,0.4,11,0.092035,34,0.415152
12,2022,0.655935,-0.208488,0.3,17,0.087555,61,0.435294
0,2020,0.924593,-0.349377,0.1,142,0.057864,393,0.464789
11,2022,0.655935,-0.208488,0.2,30,0.056807,105,0.417778
1,2020,0.924593,-0.349377,0.2,80,0.045793,207,0.429792
15,2023,0.498845,-0.188674,0.1,160,0.045396,159,0.390625
16,2023,0.498845,-0.188674,0.2,94,0.045351,84,0.382979
18,2023,0.498845,-0.188674,0.4,36,0.042308,31,0.375000
3,2020,0.377619,-0.246566,0.4,23,0.038361,46,0.358696
17,2023,0.498845,-0.188674,0.3,58,0.036425,46,0.387931


In [13]:
max_returns = []
min_returns = []
starts = []
return_thresholds = []
stock_list_len = []
mean_returns = []
trade_totals = []
mean_winrates = []
final_negative = pd.DataFrame()
for start in [2020, 2021, 2022, 2023]:
    df_final = pd.read_csv(f"backtest_by_year/{strat}_{start}_{start+1}.csv")
    df_outsample = pd.read_csv(f"backtest_by_year/{strat}_{start+1}_{start+2}.csv")
    for return_threshold in [-0.1, -0.2, -0.3, -0.4, -0.5]:
        return_1 = list(df_final[df_final['return_total'] < return_threshold]['ticker'])

        return_5_avg = df_outsample[df_outsample['ticker'].isin(return_1)]
        
        max_return = np.max(return_5_avg['return_total'])
        min_return = np.min(return_5_avg['return_total'])
        mean_return = np.mean(return_5_avg['return_total'])
        mean_winrate = np.mean(return_5_avg['winrate'])
        trade_total = np.sum(return_5_avg['trade_total'])
        max_returns.append(max_return)
        min_returns.append(min_return)
        return_thresholds.append(return_threshold)
        starts.append(start)
        stock_list_len.append(len(return_1))
        mean_returns.append(mean_return)
        trade_totals.append(trade_total)
        mean_winrates.append(mean_winrate)
final_negative['starts'] = starts
final_negative['max_returns'] = max_returns
final_negative['min_returns'] = min_returns
final_negative["return_thresholds"] = return_thresholds
final_negative["stock_list_len"] = stock_list_len
final_negative["mean_returns"] = mean_returns
final_negative["trade_totals"] = trade_totals
final_negative["mean_winrates"] = mean_winrates
final_negative.sort_values(by='mean_returns', ascending=False)

,starts,max_returns,min_returns,return_thresholds,stock_list_len,mean_returns,trade_totals,mean_winrates
16,2023,0.595596,0.000000,-0.2,12,0.103861,8,0.583333
10,2022,0.793453,-0.319805,-0.1,106,0.069214,406,0.449102
15,2023,0.595596,-0.165041,-0.1,66,0.068430,68,0.553030
11,2022,0.610445,-0.222885,-0.2,22,0.051823,91,0.419697
0,2020,0.435833,-0.284207,-0.1,64,0.020576,221,0.467187
4,2020,0.010370,0.010370,-0.5,1,0.010370,4,0.500000
17,2023,0.000000,0.000000,-0.3,1,0.000000,0,0.000000
6,2021,0.463570,-0.232791,-0.2,21,-0.009711,58,0.360317
1,2020,0.369838,-0.208307,-0.2,19,-0.010781,71,0.383333
7,2021,0.216300,-0.232791,-0.3,6,-0.029099,16,0.222222


In [45]:
return_5_avg['return_total']

Series([], Name: return_total, dtype: float64)